In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
select 
    year(date) as Year, VALUE as PCE
from 
    financial__economic_essentials.cybersyn.FINANCIAL_FRED_TIMESERIES
where true
    and variable_name = 'Personal Consumption Expenditures: Chain-type Price Index, Seasonally adjusted, Monthly, Index 2017=100'
    and date >= '1972-01-01'
    and month(date) = 1
order by date
;

# 解法１
PandasとScikit-learnライブラリによる線形回帰

In [ ]:
from sklearn.linear_model import LinearRegression

pd_df_pce_year = sql_get_data.to_pandas()

x = pd_df_pce_year["YEAR"].to_numpy().reshape(-1,1)
y = pd_df_pce_year["PCE"].to_numpy()

model = LinearRegression().fit(x, y)

In [ ]:
# test model for 2022
predictYear = 2022
pce_pred = model.predict([[predictYear]])
# print the last 5 years
st.dataframe(pd_df_pce_year.tail(), hide_index=True)
# run the prediction for 2022
print ('Prediction for '+str(predictYear)+': '+ str(round(pce_pred[0],2)))

# 解法２
Snowpark DataFrameとSnowpark MLライブラリによる線形回帰

In [ ]:
from snowflake.ml.modeling.linear_model.linear_regression import LinearRegression

snow_df_pce_year = sql_get_data.to_df()

regr = LinearRegression(
    input_cols = "YEAR",
    label_cols = "PCE",
)

_ = regr.fit(snow_df_pce_year)

In [ ]:
def predict_pce(years: list)->list:
    input = session.create_dataframe(years, schema = ["YEAR"])
    pce_pred = regr.predict(input)

    return pce_pred.to_pandas()
    
# test model for 2022
predictYear = 2022
predicted_pce = predict_pce([predictYear])

# print the last 5 years
st.dataframe (snow_df_pce_year.to_pandas().tail(), hide_index=True)
# run the prediction for 2022
print ('Prediction for '+str(predictYear)+': '+ str(round(predicted_pce.iloc[0,1],2)))

## Wrap up
今回、Snowpark MLライブラリを使って線形回帰を行ってみました。この方法のメリットは何でしょうか？

それは、「Snowflakeからデータが外に出ないこと」です。これにより、**モデルの学習時に比較的小さいローカルリソースを使うのではなく、Snowflakeのスケーラブルで従量制のコンピューティングリソースを活用できます**。
また、今後、NotebooksやSiSは、ウェアハウス分離機能によりフロントはXSのマネージドウェアハウスを、クエリ実行は任意のサイズのウェアハウスをそれぞれ使い分けることができるようになります。このとき、大容量データをフロント用のマネージドウェアハウス上でPandas・Scikit-learnを使って学習させるのではなく、Snowpark DataFrame・Snowpark MLを使って学習させることで、コスト効率良く、フレキシブルにSnowflakeのコンピューティングを活用することができます。

なお、今回の線形回帰は対象外ですが、Snowpark MLの前処理やモデリング関数の一部は、Snowflakeのウェアハウス上で分散実行されるように設計されています。こうしたSnowflakeに最適になるように構成されていることも、Snowpark MLライブラリの素晴らしいポイントだと考えております。詳しい対応リストはこちらの公式ドキュメントを参照してみてください。
https://docs.snowflake.com/developer-guide/snowpark-ml/modeling#snowpark-ml-modeling-classes

# Streamlitによる可視化

In [ ]:
import plotly.express as px

predictYears = list(range(1972, 2031))
predicted_pce = predict_pce(predictYears)

fig = px.line(snow_df_pce_year, x="YEAR", y="PCE")
fig.add_traces(px.scatter(predicted_pce, x="YEAR", y="OUTPUT_PCE").data)
st.plotly_chart(fig)

if st.button(":snowflake:"):
    st.snow()